## Import Dependencies 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import lxml.html as lh
from sklearn.decomposition import PCA
from sklearn import preprocessing
%matplotlib inline
plt.style.use('fivethirtyeight')
pd.set_option('display.max_columns', None)
import os
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import neighbors
from sklearn.neural_network import MLPClassifier
import sklearn.metrics as metrics
from sklearn.model_selection import KFold
from operator import itemgetter
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

# Get Data

List of NBA Teams

In [2]:
TEAMS_ID_NAME=[['1610612737', 'ATL'],
 ['1610612738', 'BOS'],
 ['1610612741', 'CHI'],
 ['1610612766', 'CHR'],
 ['1610612739', 'CLE'],
 ['1610612742', 'DAL'],
 ['1610612743' ,'DEN'],
 ['1610612765' ,'DET'],
 ['1610612744', 'GSW'],
 ['1610612745' ,'HOU'],
 ['1610612754' ,'IND'],
 ['1610612746' ,'LAC'],
 ['1610612747' ,'LAL'],
 ['1610612763' ,'MEM'],
 ['1610612748' ,'MIA'],
 ['1610612749' ,'MIL'],
 ['1610612750' ,'MIN'],
 ['1610612751' ,'NJN'],
 ['1610612740' ,'NOH'],
 ['1610612752' ,'NYK'],
 ['1610612753' ,'ORL'],
 ['1610612755' ,'PHI'],
 ['1610612756' ,'PHO'],
 ['1610612757' ,'POR'],
 ['1610612758' ,'SAC'],
 ['1610612759' ,'SAS'],
 ['1610612760' ,'SEA'],
 ['1610612761' ,'TOR'],
 ['1610612762' ,'UTA'],
 ['1610612764' ,'WAS']]

In [3]:
# take out CHR Team (ID 1610612766) as there is no season data for it in 2003
# Team ID
team_ID = [1610612737, 1610612738, 1610612741, 1610612739, 1610612742, 1610612743, 1610612765, 1610612744, 1610612745, 1610612754, 1610612746, 1610612747, 1610612763, 1610612748, 1610612749, 1610612750, 1610612751, 1610612740, 1610612752, 1610612753, 1610612755, 1610612756, 1610612757, 1610612758, 1610612759, 1610612760, 1610612761, 1610612762, 1610612764]

##Team Season Data

In [5]:
d = os.path.join("/content/team_season.txt")
team_season = np.loadtxt(d, delimiter=',', dtype='str')
# get header
header = team_season[0]
# remove first row, header
team_season = team_season[1:]
# convert it to an CSV file
pd.DataFrame(team_season).to_csv("team_stats.csv", header = header,index=False)

In [6]:
col_list = ["team", "year",'o_ftm','o_reb' ,'o_asts' ,'o_pf' ,'o_stl' ,'o_to' ,'o_blk', 'o_3pm','o_pts',
 'd_ftm' ,'d_reb', 'd_asts' ,'d_pf',
 'd_stl', 'd_to', 'd_blk','d_3pm','d_pts','won','lost']

team_stats = pd.read_csv("team_stats.csv",usecols=col_list)

#get team stats for 2002
stats_df = team_stats.loc[team_stats["year"] == 2003]
stats_df.head()

,team,year,o_ftm,o_reb,o_asts,o_pf,o_stl,o_to,o_blk,o_3pm,o_pts,d_ftm,d_reb,d_asts,d_pf,d_stl,d_to,d_blk,d_3pm,d_pts,won,lost
1128,ATL,2003,1534,3503,1648,1826,627,1278,408,419,7611,1574,3533,1803,1699,731,1086,409,482,7992,28,54
1129,BOS,2003,1572,3291,1683,1837,770,1289,331,553,7811,1587,3594,1882,1819,719,1334,407,555,7930,36,46
1130,CHI,2003,1330,3567,1793,1911,659,1249,394,429,7355,1673,3693,1890,1684,694,1190,442,439,7876,23,59
1131,CLE,2003,1528,3737,1808,1742,585,1166,537,247,7619,1491,3444,1783,1655,635,1018,434,431,7834,35,47
1132,DAL,2003,1475,3712,1963,1609,656,965,437,507,8626,1462,3571,1937,1689,582,1178,382,534,8262,52,30


In [7]:
TeamStats_2003 = np.array(stats_df)
#skip first 2 columns (team name and year)
team_stats = TeamStats_2003[:,2 :] 

## Game Data

In [9]:
col_list =["HOME_TEAM_ID","VISITOR_TEAM_ID","SEASON", "HOME_TEAM_WINS"]
game_data = pd.read_csv("games.csv", usecols=col_list)
# get games only for season 2004
game_data = game_data.loc[game_data["SEASON"] == 2004]


# take CHR Game data out 
game_data = game_data.loc[game_data["HOME_TEAM_ID"] != 1610612766]
game_data = game_data.loc[game_data["VISITOR_TEAM_ID"] != 1610612766]
game_data.head()

,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,HOME_TEAM_WINS
13086,1610612759,1610612765,2004,1
13087,1610612759,1610612765,2004,0
13088,1610612765,1610612759,2004,0
13089,1610612765,1610612759,2004,1
13090,1610612765,1610612759,2004,1


In [10]:
# convert into a np array
game_data = np.array(game_data)

# Create the Feature Vector

In [11]:
def createDataSet(game_data,usePCA,isNormalized):
  pca = PCA(n_components=18)
  normalizer = preprocessing.MinMaxScaler()
  data = []
  for i in range(len(game_data)):

    # find location home_team stats
    find_index = np.where(team_ID == game_data[i][0])
    index = find_index[0][0]
    # get home_team stats
    home_side = np.array(team_stats[index])

  
    # get away_team stats
    find_index = np.where(team_ID == game_data[i][1])
    index = find_index[0][0]
    away_side =  np.array(team_stats[index])


    # concatenate home_team stats and away_team stats
    feature_vector = np.concatenate((home_side, away_side), axis=None)

    
    # add fearure vector to data
    data.append(feature_vector)

  #NORMALIZE
  if (isNormalized):
    data = normalizer.fit_transform(data)

  #reduce feature vector with PCA
  if (usePCA):
    data = pca.fit_transform(data)

  ndata = []

  for z in range(len(game_data)):
    result = []
    result.append(game_data[z][3])

    # create feature vector by appending the outcome labled result of the game  
    feature_vector = np.concatenate((data[z],result), axis=None)
    ndata.append(feature_vector)

  data = np.array(ndata)
  
  return data

# ML Techniques

Scoring matrix 

In [12]:
def scores(model):
    
    model.fit(xtrain, ytrain)
    y_pred = model.predict(xtest)
    
    print("Accuracy score: %.3f" % metrics.accuracy_score(ytest, y_pred))
    print("Recall: %.3f" % metrics.recall_score(ytest, y_pred))
    print("Precision: %.3f" % metrics.precision_score(ytest, y_pred))
    print("F1: %.3f" % metrics.f1_score(ytest, y_pred))
    
    proba = model.predict_proba(xtest)
    print("Log loss: %.3f" % metrics.log_loss(ytest, proba))

    pos_prob = proba[:, 1]
    print("Area under ROC curve: %.3f" % metrics.roc_auc_score(ytest, pos_prob))
    
    cv = cross_val_score(model, xtest, ytest, cv = 10, scoring = 'accuracy')
    print("Accuracy (cross validation score): %0.3f (+/- %0.3f)" % (cv.mean(), cv.std() * 2))

    print("confusion matrix")
    print( metrics.confusion_matrix(ytest, y_pred) )
    
    return y_pred

## Training and testing our models

Get the different Data Sets

In [13]:
#Original data
dA = createDataSet(game_data,False,False)
#PCA with Normalized Data
dB = createDataSet(game_data,True,True)
#PCA Data
dC = createDataSet(game_data,True,False)
#Normalized Data
dD = createDataSet(game_data,False,True)

**Results using Original Data (NO PCA or Normalization)**

In [15]:
trainA, testA = train_test_split( dA, test_size = 0.25,random_state = 20)

xtrain = trainA[:, :-1] # for all but last column
ytrain = trainA[:, -1].astype(int) # for last column (result label )

xtest = testA[:, :-1] 
ytest = testA[:, -1].astype(int) 

print("Training set size: %.0f" % len(xtrain))
print("Testing set size: %.0f" % len(xtest))
print()


print("SVM STATS")
#svc = SVC(probability=True)
svc = SVC(kernel='rbf',C=100,probability=True)
y_svc = scores(svc)
print()

print("knn STATS")
knn = neighbors.KNeighborsClassifier(n_neighbors = 12, weights = 'uniform')
y_knn = scores(knn)
print()

print("logreg STATS")
logreg = LogisticRegression(max_iter=10000)
y_log = scores(logreg)

Training set size: 957
Testing set size: 320

SVM STATS
Accuracy score: 0.644
Recall: 0.995
Precision: 0.634
F1: 0.775
Log loss: 0.621
Area under ROC curve: 0.726
Accuracy (cross validation score): 0.656 (+/- 0.097)
confusion matrix
[[ 10 113]
 [  1 196]]

knn STATS
Accuracy score: 0.662
Recall: 0.736
Precision: 0.721
F1: 0.729
Log loss: 0.620
Area under ROC curve: 0.677
Accuracy (cross validation score): 0.650 (+/- 0.104)
confusion matrix
[[ 67  56]
 [ 52 145]]

logreg STATS
Accuracy score: 0.678
Recall: 0.832
Precision: 0.701
F1: 0.761
Log loss: 0.595
Area under ROC curve: 0.712
Accuracy (cross validation score): 0.662 (+/- 0.145)
confusion matrix
[[ 53  70]
 [ 33 164]]


**Results using PCA Data**

In [17]:
trainA, testA = train_test_split( dC, test_size = 0.25,random_state = 20)

xtrain = trainA[:, :-1] # for all but last column
ytrain = trainA[:, -1].astype(int) # for last column (result label )

xtest = testA[:, :-1] 
ytest = testA[:, -1].astype(int) 


print("SVM STATS")
#svc = SVC(probability=True)
svc = SVC(kernel='rbf',probability=True)
y_svc = scores(svc)
print()

print("knn STATS")
knn = neighbors.KNeighborsClassifier(n_neighbors = 18, weights = 'uniform')
y_knn = scores(knn)
print()

print("logreg STATS")
logreg = LogisticRegression(max_iter=10000)
y_log = scores(logreg)

SVM STATS
Accuracy score: 0.684
Recall: 0.853
Precision: 0.700
F1: 0.769
Log loss: 0.599
Area under ROC curve: 0.744
Accuracy (cross validation score): 0.669 (+/- 0.070)
confusion matrix
[[ 51  72]
 [ 29 168]]

knn STATS
Accuracy score: 0.675
Recall: 0.797
Precision: 0.710
F1: 0.751
Log loss: 0.599
Area under ROC curve: 0.718
Accuracy (cross validation score): 0.684 (+/- 0.152)
confusion matrix
[[ 59  64]
 [ 40 157]]

logreg STATS
Accuracy score: 0.650
Recall: 0.802
Precision: 0.684
F1: 0.738
Log loss: 0.603
Area under ROC curve: 0.710
Accuracy (cross validation score): 0.688 (+/- 0.097)
confusion matrix
[[ 50  73]
 [ 39 158]]


**Results using Normalized Data**

In [18]:
trainA, testA = train_test_split( dD, test_size = 0.25,random_state = 20)

xtrain = trainA[:, :-1] # for all but last column
ytrain = trainA[:, -1].astype(int) # for last column (result label )

xtest = testA[:, :-1] 
ytest = testA[:, -1].astype(int) 


print("SVM STATS")
#svc = SVC(probability=True)
svc = SVC(kernel='rbf',C=100,probability=True)
y_svc = scores(svc)
print()

print("knn STATS")
knn = neighbors.KNeighborsClassifier()
y_knn = scores(knn)
print()

print("logreg STATS")
logreg = LogisticRegression(max_iter=10000)
y_log = scores(logreg)

SVM STATS
Accuracy score: 0.606
Recall: 0.701
Precision: 0.673
F1: 0.687
Log loss: 0.635
Area under ROC curve: 0.654
Accuracy (cross validation score): 0.647 (+/- 0.192)
confusion matrix
[[ 56  67]
 [ 59 138]]

knn STATS
Accuracy score: 0.597
Recall: 0.695
Precision: 0.665
F1: 0.680
Log loss: 1.894
Area under ROC curve: 0.611
Accuracy (cross validation score): 0.669 (+/- 0.113)
confusion matrix
[[ 54  69]
 [ 60 137]]

logreg STATS
Accuracy score: 0.684
Recall: 0.863
Precision: 0.697
F1: 0.771
Log loss: 0.605
Area under ROC curve: 0.708
Accuracy (cross validation score): 0.653 (+/- 0.154)
confusion matrix
[[ 49  74]
 [ 27 170]]


**Results using PCA with Normalized Data**

In [ ]:
trainA, testA = train_test_split( dB, test_size = 0.25,random_state = 20)

xtrain = trainA[:, :-1] # for all but last column
ytrain = trainA[:, -1].astype(int) # for last column (result label )

xtest = testA[:, :-1] 
ytest = testA[:, -1].astype(int) 


print("SVM STATS")
#svc = SVC(probability=True)
svc = SVC(kernel='rbf',probability=True)
y_svc = scores(svc)
print()

print("knn STATS")
knn = neighbors.KNeighborsClassifier(n_neighbors = 12, weights = 'uniform')
y_knn = scores(knn)
print()

print("logreg STATS")
logreg = LogisticRegression(max_iter=10000)
y_log = scores(logreg)

SVM STATS
Accuracy score: 0.706
Recall: 0.843
Precision: 0.725
F1: 0.779
Log loss: 0.607
Area under ROC curve: 0.717
Accuracy (cross validation score): 0.650 (+/- 0.121)
confusion matrix
[[ 60  63]
 [ 31 166]]

knn STATS
Accuracy score: 0.637
Recall: 0.701
Precision: 0.708
F1: 0.704
Log loss: 0.611
Area under ROC curve: 0.685
Accuracy (cross validation score): 0.628 (+/- 0.099)
confusion matrix
[[ 66  57]
 [ 59 138]]

logreg STATS
Accuracy score: 0.672
Recall: 0.848
Precision: 0.690
F1: 0.761
Log loss: 0.618
Area under ROC curve: 0.687
Accuracy (cross validation score): 0.653 (+/- 0.081)
confusion matrix
[[ 48  75]
 [ 30 167]]
